In [1]:
%load_ext autoreload
%autoreload 

In [2]:
from result_saver import SaverProvider, metadata_helper, metadata_loader
from soft_info.UnionFind import get_counts
import pandas as pd

# Generate data

In [3]:
provider = SaverProvider()
backend = provider.get_backend('ibmq_jakarta')

In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile

qc = QuantumCircuit(1, 1)
#qc.x(0)
qc.measure(0, 0)
print(qc.draw(fold=-1))

initial_layout = [6]
transpiled_qc = transpile(qc, backend, initial_layout=initial_layout)

print(transpiled_qc.draw(fold=-1))

     ┌─┐
  q: ┤M├
     └╥┘
c: 1/═╩═
      0 
                  
ancilla_0 -> 0 ───
                  
ancilla_1 -> 1 ───
                  
ancilla_2 -> 2 ───
                  
ancilla_3 -> 3 ───
                  
ancilla_4 -> 4 ───
                  
ancilla_5 -> 5 ───
               ┌─┐
      q_0 -> 6 ┤M├
               └╥┘
          c: 1/═╩═
                0 


In [8]:
job_metadata = metadata_helper(n_shots=1e6, meas_level=1, sampled_state=0, num_qubits=1)
job_id = backend.run(job_metadata, transpiled_qc, shots=1e6, meas_level=1, meas_return='single').job_id() 

Custom Warning: updating metadata



In [5]:
job_metadata = metadata_loader()
memories = {}
for i, job_id in enumerate(job_metadata[job_metadata['notebook_name'] == 'KDE_Jak']['job_id']):
    memory_name = 'memory_' + str((i + 1) % 2) + "_state"
    memories[memory_name] = provider.retrieve_job(job_id).result().get_memory()
memories

{'memory_1_state': array([[38845461.-36435344.j],
        [19906178.-41771744.j],
        [10636813.-47942273.j],
        ...,
        [15644804.-45668296.j],
        [38471320.-35815594.j],
        [16732111.-29547949.j]]),
 'memory_0_state': array([[-47030367.-56241506.j],
        [-30773028.-51585687.j],
        [-70253546.-60719018.j],
        ...,
        [-34120294.-81646139.j],
        [-38576581.-35055936.j],
        [ -6236616.-51748192.j]])}

# KDE Fitting